In [1]:
### 0. 임포트
# API 호출 관련
import bs4
import json
import requests
import psycopg2
from os import name
import numpy  as np
import pandas as pd
from tqdm import tqdm

# 날짜 관련
import datetime
from datetime import date
from dateutil.relativedelta import relativedelta

### 최종

In [2]:
### 0. 임포트
# API 호출 관련
import bs4
import json
import requests
import psycopg2
from os import name
import numpy  as np
import pandas as pd
from tqdm import tqdm

# 날짜 관련
import datetime
from datetime import date
from dateutil.relativedelta import relativedelta

### 1. 시군구코드 API
lawd_cd_df = pd.DataFrame()
lawd_cd_list = []
lawd_cd_unique = []

for i in range(1, 22):
    
    url = 'http://apis.data.go.kr/1741000/StanReginCd/getStanReginCdList'
    serviceKey = 'C94PKnigawlH2iISVWz1Itw9tVzyuOCbOvKgzCrGOUZWvs3XqrkhH+/ntypJ7kK2P2g16vLYCJif3TNfO1i1KA=='
    params = {
        'serviceKey' : serviceKey,
        'pageNo' : str(i),
        'numOfRows' : '1000',
        'type' : 'json'
    }
    response = requests.get(url, params=params)
    content = response.text
    
    # Json List 변환
    lawd_cd_df = pd.DataFrame(pd.json_normalize(data = json.loads(content), record_path = 'StanReginCd')['row'][1])
    lawd_cd_list = list(lawd_cd_df['locatjumin_cd'].str[:10])
    
    # 시군구코드 unique하게 남기기
    for x in lawd_cd_list:
        if x not in lawd_cd_unique:
            lawd_cd_unique.append(x)

### 2. 건축인허가 기본개요 조회 API 호출
# 변수 생성
row_list   = [] # 행 값
name_list  = [] # 열이름 값
value_list = [] # 데이터 값
DF = pd.DataFrame() # 최종 값
table = 'm1.rtms_trade' # Table
month = (date.today() - relativedelta(months = 1)).strftime('%Y%m%d') # 전월
url = 'http://openapi.molit.go.kr/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcNrgTrade'
serviceKey = 'C94PKnigawlH2iISVWz1Itw9tVzyuOCbOvKgzCrGOUZWvs3XqrkhH+/ntypJ7kK2P2g16vLYCJif3TNfO1i1KA=='

for code in tqdm(lawd_cd_unique):
    params = {
          'serviceKey' : serviceKey
        , 'sigunguCd'  : lawd_cd_unique[0][:5]
        , 'bjdongCd'   : lawd_cd_unique[0][5:]
        , 'startDate'  : month
        , 'numOfRows'  : '1000000'
        , 'pageNo'  : '1'
    }
    response = requests.get(url, params=params)
    content  = response.text

    # item 태그 분리
    xml_obj = bs4.BeautifulSoup(content,'lxml-xml')
    rows    = xml_obj.findAll('row')

    # 데이터 수집
    for i in range(0, len(rows)):
        columns = rows[i].find_all()

        for j in range(0,len(columns)):
            name_list.append(columns[j].name)
            value_list.append(columns[j].text)

        row_list = value_list
        df_each  = pd.DataFrame([row_list], columns=name_list)
        DF = pd.concat([DF, df_each]).reset_index(drop = True)

        # 초기화
        name_list  = []
        value_list = []

# 전처리
DF = DF.replace(' ', None)
DF = DF.replace(np.nan, None)
if '구분' in DF.columns:
    DF = DF.drop(['구분'], axis = 1)

# rename
DF = DF.rename(columns = {"거래금액":"deal_amount",
                          "거래유형":"trade_type",
                          "건물면적":"building_area",
                          "건물주용도":"building_use",
                          "건축년도":"build_year",
                          "년":"deal_year",
                          "대지면적":"plottage",
                          "법정동":"dong",
                          "시군구":"sigungu",
                          "용도지역":"land_use",
                          "월":"deal_month",
                          "유형":"building_type",
                          "일":"deal_day",
                          "중개사소재지":"broker_location",
                          "지역코드":"regional_code",
                          "층":"floor_",
                          "해제사유발생일":"unregister_day",
                          "해제여부":"unregister_status"})

# dtype 변경
DF['deal_amount'] = DF['deal_amount'].apply(lambda x: x.replace(',', '')).astype('int64')

### 3. DB INSERT
# execute_mogrify
def execute_mogrify(conn, df, table, val):
    # Create a list of tuples from the dataframe values
    tuples = [tuple(x) for x in df.to_numpy()]
    # Comma-separated dataframe columns
    cols = ','.join(list(df.columns))
    # SQL query to execute
    cursor = conn.cursor()
    values = [cursor.mogrify(val, tup).decode('utf8') for tup in
              tuples]
    query = "INSERT INTO %s(%s) VALUES " % (table, cols) + ",".join(values)

    try:
        cursor.execute(query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    cursor.close()

# values 설정
val = '(' + ('%s,' * len(DF.columns))[:-1] + ')'

# insert
for j in range(0, len(DF), 10000):

    # DB Connect
    conn = psycopg2.connect(
        host='redshift-cluster-1.ctvbwnnvbdkl.ap-northeast-2.redshift.amazonaws.com',
        port=5439,
        dbname='dev',
        user='awsuser',
        password='cremaoAdmin1234qwer!!'
    )

    # 10000개 단위로 나누기
    tmp = DF[j:j + 10000]

    # insert
    execute_mogrify(conn, tmp, table, val)
    print(datetime.datetime.now(), ' : ', j)
    tmp = pd.DataFrame()

# row count
print(len(DF))

SSLError: HTTPSConnectionPool(host='apis.data.go.kr', port=443): Max retries exceeded with url: /1613000/ArchPmsService_v2/getApBasisOulnInfo?serviceKey=sJuqUegKT1Q0R4IRDqoJklMK8xDxru%2BrNu%2BK5JXRh1mbeH4tFTWwQYWVK3xdbbJCCe14u5bQyl43S28%2F5TtfrA%3D%3D&pageNo=1&numOfRows=1000&type=json (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1131)')))

### test

In [4]:
### 1. 변수 생성
# Table
table = 'm1.rtms_trade'

# 시군구코드 API
lawd_cd_df = pd.DataFrame()
lawd_cd_list = []
lawd_cd_unique = []

for i in range(1, 22):
    
    url = 'http://apis.data.go.kr/1741000/StanReginCd/getStanReginCdList'
    serviceKey = 'C94PKnigawlH2iISVWz1Itw9tVzyuOCbOvKgzCrGOUZWvs3XqrkhH+/ntypJ7kK2P2g16vLYCJif3TNfO1i1KA=='
    params = {
        'serviceKey' : serviceKey,
        'pageNo' : str(i),
        'numOfRows' : '1000',
        'type' : 'json'
    }
    response = requests.get(url, params=params)
    content = response.text
    
    # Json List 변환
    lawd_cd_df = pd.DataFrame(pd.json_normalize(data = json.loads(content), record_path = 'StanReginCd')['row'][1])
    lawd_cd_list = list(lawd_cd_df['locatjumin_cd'].str[:10])
    
    # 시군구코드 unique하게 남기기
    for x in lawd_cd_list:
        if x not in lawd_cd_unique:
            lawd_cd_unique.append(x)

In [19]:
### 2. 건축인허가 기본개요 조회 API 호출
# 변수 생성
row_list   = [] # 행 값
name_list  = [] # 열이름 값
value_list = [] # 데이터 값
DF = pd.DataFrame() # 최종 값
# table = 'm1.rtms_trade' # Table
month = (date.today() - relativedelta(months = 1)).strftime('%Y%m') + '01' # 전월
url = 'http://openapi.molit.go.kr/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcNrgTrade'
serviceKey = 'C94PKnigawlH2iISVWz1Itw9tVzyuOCbOvKgzCrGOUZWvs3XqrkhH+/ntypJ7kK2P2g16vLYCJif3TNfO1i1KA=='

for code in tqdm(lawd_cd_unique):
    params = {
          'serviceKey' : serviceKey
        , 'sigunguCd'  : lawd_cd_unique[0][:5]
        , 'bjdongCd'  : lawd_cd_unique[0][5:]
        , 'startDate'  : month
        , 'numOfRows'  : '1000000'
        , 'pageNo'  : '1'
    }
    response = requests.get(url, params=params)
    content  = response.text

    # item 태그 분리
    xml_obj = bs4.BeautifulSoup(content,'lxml-xml')
    rows    = xml_obj.findAll('row')

    # 데이터 수집
    for i in range(0, len(rows)):
        columns = rows[i].find_all()

        for j in range(0,len(columns)):
            name_list.append(columns[j].name)
            value_list.append(columns[j].text)

        row_list = value_list
        df_each  = pd.DataFrame([row_list], columns=name_list)
        DF = pd.concat([DF, df_each]).reset_index(drop = True)

        # 초기화
        name_list  = []
        value_list = []

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 20550/20550 [28:31<00:00, 12.00it/s]


In [20]:
DF

""


In [46]:
DF = DF.replace('',np.nan)

In [47]:
DF = DF.replace(' ',np.nan)

In [21]:
DF['sido_cd'].value_counts()

KeyError: 'sido_cd'

In [49]:
DF

,region_cd,sido_cd,sgg_cd,umd_cd,ri_cd,locatjumin_cd,locatjijuk_cd,locatadd_nm,locat_order,locat_rm,locathigh_cd,locallow_nm,adpt_de
0,2717010900,27,170,109,00,2717010900,2717010900,대구광역시 서구 원대동3가,9,NaN,2717000000,원대동3가,NaN
1,2720000000,27,200,000,00,2720000000,2720000000,대구광역시 남구,4,NaN,2700000000,남구,NaN
2,2717010700,27,170,107,00,2717010700,2717010700,대구광역시 서구 원대동1가,7,NaN,2717000000,원대동1가,NaN
3,2717010800,27,170,108,00,2717010800,2717010800,대구광역시 서구 원대동2가,8,NaN,2717000000,원대동2가,NaN
4,3020015200,30,200,152,00,3020015200,3020015200,대전광역시 유성구 둔곡동,52,NaN,3020000000,둔곡동,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2620010400,26,200,104,00,2620010400,2620010400,부산광역시 영도구 대평동2가,4,NaN,2620000000,대평동2가,NaN
96,2620010500,26,200,105,00,2620010500,2620010500,부산광역시 영도구 남항동1가,5,NaN,2620000000,남항동1가,NaN
97,2620010600,26,200,106,00,2620010600,2620010600,부산광역시 영도구 남항동2가,6,NaN,2620000000,남항동2가,NaN
98,2620010700,26,200,107,00,2620010700,2620010700,부산광역시 영도구 남항동3가,7,NaN,2620000000,남항동3가,NaN
